In [1]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [94]:
import pandas as pd
import numpy as np

In [95]:
df = pd.read_csv('menu_concated_v1.3')
df = df.dropna()

In [96]:
menu = df['menu'].tolist()
menu = list(set(menu))

In [97]:
from konlpy.tag import Okt
from tqdm import tqdm
okt = Okt()

In [165]:
menu_token = [okt.morphs(i) for i in tqdm(menu)]

100%|██████████| 51437/51437 [03:00<00:00, 284.24it/s]


In [166]:
len(menu_token)

51437

In [167]:
menu_token = [j for i in menu_token for j in i]
menu_token[:10]

['안주', '탕', '씨', '푸드', '갈릭', '레몬', '스무디', '딸기스무디', '딸기', '생크림']

In [168]:
def to_ngrams(words, n):
  ngrams = []
  for b in range(0, len(words) - n + 1):
      ngrams.append(words[b:b+n])
  return ngrams

In [169]:
menu_ngrams = []

In [170]:
menu_ngrams = menu_token
menu_ngrams += [to_ngrams(i, 2) for i in menu]
menu_ngrams += [to_ngrams(i, 3) for i in menu]
menu_ngrams += [to_ngrams(i, 4) for i in menu]
menu_ngrams += [to_ngrams(i, 5) for i in menu]
menu_ngrams += [to_ngrams(i, 6) for i in menu]

In [171]:
menu_ngrams = [j for i in menu_ngrams for j in i]

In [172]:
len(menu_ngrams)

1176675

In [173]:
menu_count = {}

for i in menu_ngrams:
  if i not in menu_count:
    menu_count[i] = 1
  else:
    menu_count[i] += 1

In [174]:
menu_count['샐러드']

552

In [175]:
items = menu_count.items()
sorted_items = sorted(items, key=lambda x: x[1], reverse=True) # 빈도 내림차순 정렬

In [181]:
sorted_items_rank = [v for v, c in sorted_items if c >= 150 and len(v) != 1] # 150개 이상 중복 + 길이가 2 이상
sorted_items_rank = sorted(sorted_items_rank, key=lambda x: len(x), reverse=True) # 길이 내림차순 정렬렬
sorted_items_rank[:10]

['아메리카노',
 '스테이크',
 '샌드위치',
 '후라이드',
 '아메리카',
 '메리카노',
 '크림치즈',
 '포테이토',
 '오리지널',
 '블루베리']

In [177]:
menu_cluster = {}

for food_token in tqdm(sorted_items_rank):
  for food_name in menu:
    if food_name not in menu_cluster: # 이미 분류된 메뉴는 건들지 않음

      # 메뉴명이 토큰명보다 길면서 동시에 토큰명으로 끝나는 메뉴를 토큰 카테고리로 분류 
      if len(food_token) <= len(food_name) and food_name[-len(food_token):] == food_token:
        menu_cluster[food_name] = food_token

100%|██████████| 1217/1217 [00:30<00:00, 39.35it/s]


In [178]:
menu_cluster

{'석류요거트스무디': '요거트스무디',
 '키위요거트스무디': '요거트스무디',
 '플레인요거트스무디': '요거트스무디',
 '복숭아요거트스무디': '요거트스무디',
 '티파니블루요거트스무디': '요거트스무디',
 '베리믹스요거트스무디': '요거트스무디',
 '유자요거트스무디': '요거트스무디',
 '딸기레몬요거트스무디': '요거트스무디',
 '더블딸기요거트스무디': '요거트스무디',
 '말차요거트스무디': '요거트스무디',
 '자몽요거트스무디': '요거트스무디',
 '생자몽요거트스무디': '요거트스무디',
 '한라봉요거트스무디': '요거트스무디',
 '망고패션콜라겐요거트스무디': '요거트스무디',
 '딸기버블요거트스무디': '요거트스무디',
 '아보카도요거트스무디': '요거트스무디',
 '밀크티버블요거트스무디': '요거트스무디',
 '레몬요거트스무디': '요거트스무디',
 '트리플베리요거트스무디': '요거트스무디',
 '키워요거트스무디': '요거트스무디',
 '망고레몬버블요거트스무디': '요거트스무디',
 '사과요거트스무디': '요거트스무디',
 '그린티버블요거트스무디': '요거트스무디',
 '블루베리요거트스무디': '요거트스무디',
 '딸기요거트스무디': '요거트스무디',
 '자두요거트스무디': '요거트스무디',
 '바나나요거트스무디': '요거트스무디',
 '애플망고요거트스무디': '요거트스무디',
 '스트로베리요거트스무디': '요거트스무디',
 '리얼딸기요거트스무디': '요거트스무디',
 '생과일요거트스무디': '요거트스무디',
 '딸기망고요거트스무디': '요거트스무디',
 '망고요거트스무디': '요거트스무디',
 '베리베리요거트스무디': '요거트스무디',
 '망고버블요거트스무디': '요거트스무디',
 '청사과요거트스무디': '요거트스무디',
 '생딸기요거트스무디': '요거트스무디',
 '깔라만시요거트스무디': '요거트스무디',
 '석류콜라겐요거트스무디': '요거트스무디',
 '수제요거트스무디': '요거트스무디',
 '천혜향요거트스무디': '요거트스무디

In [179]:
df_menu = pd.DataFrame(columns=['title', 'category'], data=menu_cluster.items())
len(df_menu['category'].unique())

801

In [180]:
df_menu.sample(50)

,title,category
24655,닭껍질무침,무침
28067,파채국물닭발,닭발
2972,와플생크림치즈추가,치즈추가
10879,인닭볶음탕당면사리,면사리
11093,카니크림고로케,고로케
13211,그린어니언치즈,치즈
22040,해물섞어찌개,찌개
32232,리제르바소비뇽블랑,블랑
14716,치킨무추가,추가
1933,고추핫양념치킨,양념치킨
